In [ ]:
#ez az alap kódod, egy néhány dolgot módosítottam, de majd szerintem látni fogod meg majd személyesen meg is beszéljük

import numpy as np
import cv2 as cv
import keyboard #nincs benne az alap anacondában lehet instalálni kell - pip install keyboard
from time import time

#üres listák tároláshoz
karakterek=[]
betuk=[]
#betuk_ford=[] #ha majd implementáljuk a fordítót akkor talán érdemes külön eltárolni és nem módosítani de ez csak javaslat
szavak=[]
#szavak_ford=[]


img=np.zeros(300*300) #fekete fehérképek
img=img.reshape(300,300)
img2=np.ones(300*300)
img2=img2.reshape(300,300)


#ezzel tudok majd kilépni
k=0


while(1):
    if (keyboard.is_pressed(' ')):
        start=time() #idő fehér kezdetekor
        while(keyboard.is_pressed(' ')): # space felengedéséig fehér
            cv.imshow('img',img2) #fehér kép
            k=cv.waitKey(1)
        end=time()  # while vége, itt a kép már nem fehér, megnézem az időt
        feher_ido=end-start #idők különbségei
        
        if(feher_ido>1): #ha 1 mp nél rövidebb akkor . egyébként - 
            karakterek.append('-')
            print('-')
        else:
            karakterek.append('.')
            print('.')
   
    else:  #ha nem történik space lenyomása 
        start_fekete=time()
        while(not keyboard.is_pressed(' ')): #amíg nincs space
            cv.imshow('img',img)
            k=cv.waitKey(1)
            # Esc-re leáll a program 
            if k==27:
                print('break')
                break
        end_fekete=time()#while vége, space lenyomva, ido megnezese
        fekete_ido=end_fekete-start_fekete #meddig volt fekete
        
        if (fekete_ido==0): #ha csak egy fekete villanás (azaz nem történt semmi) azt figyelmn kívül hagyja
            continue
            
        elif(fekete_ido <=1): #rövid szünet 0mp < szünet <= 1mp - gondolkodom, hogy egy <1.5 nem lenne e jobb
            print('jelköz')
            
        elif(fekete_ido<=3):#1mp < szünet <=3 mp, beolvasott karakterek morse betűvé összegzése, szavakba beillesztés  
            betuk.append(karakterek) #az eddigi morse karaktereket beteszi egy betűbe
            # itt kell majd fordítani
            # betuk_ford.append(fordito_függvény(karakterek))
            print(karakterek,'a legfrissebb (morse) betu')
            
            #újabb morse betű kereséséhez
            karakterek=[] #üres karaktereklista 
        
        else: #3mp < szünet (<=7)
            betuk.append(karakterek) #betűkhöz hasonlóan járunk el, karakterek->betuk(->fordítás)->szavak
            # itt kell majd fordítani
            #betuk_ford.append(fordito_függvény(karakterek))
            szavak.append(betuk)
            #szavak_ford.append(betuk_ford) #implementálás után
            print(betuk,'a legfrissebb (morse) szo')
            
            #új szó kereséséhez
            betuk=[]
            karakterek=[]
         
        #én azt néztem, hogy a morse csak karakterek illetve szavak közötti szünetet értelmez és ha belegondolsz
        #csak a saját helyzeteden nehezítesz ha elkezdel mondatokkal is foglalkozni, mert amugy sem különböztet meg a morse
        #kis és nagybetűket illetve ha egy szó végére még beszúrnak egy pontot vagy felkiáltó jelet máris lehet látni
        #hogy egy mondatnak hol van vége ill. eleje szóval csak azért töröltem azt a részt de beszélhetünk még róla
        #ha van racionális ok a megtartása mellett ;)

        #ez az Esc re breakelés folytatása tt léepk ki a második ciklusbóé
        #azértkell az elifek után hogy az utolsó betu/szó/mondatis hozzá legyen fűzve a listához
        if k==27:
            break
print(szavak)
#print(szavak_ford) #majdmajd
cv.destroyAllWindows()

In [ ]:
#itt meg már megpróbáltam beleimplementálni a videós, fényes részét is a dolognak de még nem sikerült tesztelni mert út közben nem olyan egyszerű

import numpy as np
import cv2 as cv #ennek megfelelően van mindenhol, kódmásolásnál kell csak figyelni a 2es a cv2ből ki legyen törölve
import keyboard #nincs benne az alap anacondában lehet instalálni kell - pip install keyboard
from time import time

#üres listák tároláshoz
karakterek=[]
betuk=[]
szavak=[]


img=np.zeros(300*300) #fekete fehérképek
img=img.reshape(300,300)
img2=np.ones(300*300)
img2=img2.reshape(300,300)

kernel = np.ones((5,5),np.uint8) #kernel az erode-hoz

def kepolvasas_szamolas():
    ret, frame = cap.read() #kamera képe
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) #kép szürkébe
    ret, frame2 = cv.threshold(gray_frame,253,255,cv.THRESH_BINARY) #kép thresholdolva
    frame_final=frame2-base_frame #alapkép kivonása, hogy az alapból benne levő fényforrások ne legyenek jelen mint zaj
    kerneled_f_frame = cv.erode(frame_final,kernel,iterations = 1) #kép erode-álása, tehát az egy-egy maradék zajpixel kivonása
    cv.imshow('frame1',kerneled_f_frame)
    frame=np.reshape(kerneled_f_frame,480*640)#itt
    value=0
    for i in np.arange(480*640):#illetve itt érdemes egy kerneled_f_frame.shape pel ránézni, hogy a kamerád milyen képarányokkal vesz fel
        if(frame[i]==255):
            value=value+1
    return value


#ezzel tudok majd kilépni
k=0

cap = cv.VideoCapture(0) #kamera
ret, frame = cap.read() #kamera képe
gray_frame_base = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) #alapkép szürkébe
ret, base_frame = cv.threshold(gray_frame_base,200,255,cv.THRESH_BINARY) #alapkép thresholdolva
val=0
while(1):
    if (kepolvasas_szamolas()>10):
        start=time() #idő fehér kezdetekor
        while(keyboard.is_pressed(' ')): # space felengedéséig fehér
            cv.imshow('img',img2) #fehér kép
            k=cv.waitKey(1)
        end=time()  # while vége, itt a kép már nem fehér, megnézem az időt
        feher_ido=end-start #idők különbségei
        
        if(feher_ido>1): #ha 1 mp nél rövidebb akkor . egyébként - 
            karakterek.append('-')
            print('-')
        else:
            karakterek.append('.')
            print('.')
   
    else: #nem történik space lenyomása - nemtudom h nem lenne e érdemes elif(val==0): jobb  
        start_fekete=time()
        while(kepolvasas_szamolas()<10): #amíg nincs space
            cv.imshow('img',img)
            k=cv.waitKey(1)
            # Esc-re leáll a program 
            if k==27:
                print('break')
                break
        end_fekete=time()#while vége, space lenyomva, ido megnezese
        fekete_ido=end_fekete-start_fekete #meddig volt fekete
        
        if (fekete_ido==0): #ha csak egy fekete villanás (azaz nem történt semmi) azt figyelmn kívül hagyja
            continue
            
        elif(fekete_ido <=1): #rövid szünet 0mp < szünet <= 1mp - gondolkodom, hogy egy <1.5 nem lenne e jobb
            print('jelköz')
            
        elif(fekete_ido<=3):#1mp < szünet <=3 mp, beolvasott karakterek morse betűvé összegzése, szavakba beillesztés  
            betuk.append(karakterek) #az eddigi morse karaktereket beteszi egy betűbe
            # itt kell majd fordítani
            
            # betuk_ford[-1]=fordito_függvény(karakterek)
            print(karakterek,'a legfrissebb (morse) betu')
            
            #újabb morse betű kereséséhez
            karakterek=[] #üres karaktereklista 
       
   
        
        else: #3mp < szünet (<=7 elméletben)
            betuk.append(karakterek) #betűkhöz hasonlóan járunk el, karakterek->betuk(->fordítás)->szavak
            # itt kell majd fordítani
            #betuk[-1]=fordito_függvény(karakterek)
            szavak.append(betuk)
            print(betuk,'a legfrissebb (morse) szo')
            
            #új szó kereséséhez
            betuk=[]
            karakterek=[]
            
            
        #ez az Esc re breakelés folytatása tt léepk ki a második ciklusbóé
        #azértkell az elifek után hogy az utolsó betu/szó/mondatis hozzá legyen fűzve a listához
        if k==27:
            break
            
print(szavak)     

cap.release()
cv.destroyAllWindows()#békés végszó